In [1]:
import pandas as pd
import numpy as np

## Cleanup Users and Repos dataframes

In [2]:
repos = pd.read_csv('data/new_subset_data/new_subset_repos.csv', sep='\t')
repos.info()
repos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188865 entries, 0 to 1188864
Data columns (total 7 columns):
id             1188865 non-null int64
url            1188865 non-null object
owner_id       1188865 non-null int64
name           1188859 non-null object
language       1188865 non-null object
created_at     1188865 non-null object
forked_from    1188865 non-null object
dtypes: int64(2), object(5)
memory usage: 63.5+ MB


,id,url,owner_id,name,language,created_at,forked_from
0,2,https://api.github.com/repos/kennethkalmer/ruo...,4,ruote-kit,Ruby,2009-06-10 20:32:21,\N
1,3,https://api.github.com/repos/matplotlib/basemap,23,basemap,C++,2011-02-19 02:58:42,\N
2,5,https://api.github.com/repos/funkaster/cocos2d-x,28,cocos2d-x,C,2012-03-12 17:48:19,6
3,6,https://api.github.com/repos/cocos2d/cocos2d-x,31,cocos2d-x,C++,2010-11-18 23:17:00,\N
4,10,https://api.github.com/repos/chapuni/llvm,67,llvm,C++,2011-02-01 09:11:48,\N


In [3]:
repos.id.sort_values().head()

0     2
1     3
2     5
3     6
4    10
Name: id, dtype: int64

In [4]:
repos.loc[repos.forked_from == '\N', 'forked_from'] = '0'

In [5]:
repos.forked_from = pd.to_numeric(repos.forked_from)

In [6]:
repos.created_at = pd.to_datetime(repos.created_at)

In [7]:
repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188865 entries, 0 to 1188864
Data columns (total 7 columns):
id             1188865 non-null int64
url            1188865 non-null object
owner_id       1188865 non-null int64
name           1188859 non-null object
language       1188865 non-null object
created_at     1188865 non-null datetime64[ns]
forked_from    1188865 non-null int64
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 63.5+ MB


In [16]:
repos.to_pickle('data/pickled/new_subset_repos.pkl')

In [8]:
users = pd.read_csv('data/new_subset_data/new_subset_users.csv', sep='\t')
users.info()
users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50008 entries, 0 to 50007
Data columns (total 9 columns):
id              50008 non-null int64
login           50008 non-null object
company         49988 non-null object
created_at      50008 non-null object
type            50008 non-null object
country_code    50008 non-null object
state           50008 non-null object
city            50008 non-null object
location        50007 non-null object
dtypes: int64(1), object(8)
memory usage: 3.4+ MB


,id,login,company,created_at,type,country_code,state,city,location
0,421183,msizov,\N,2012-07-28 09:25:36,USR,\N,\N,\N,\N
1,8711566,Holy87,\N,2014-06-20 14:09:22,USR,\N,\N,\N,Neaples
2,1523784,ghswa,\N,2012-11-30 13:14:01,USR,gb,\N,\N,UK
3,5553551,yienren,\N,2013-04-13 12:01:22,USR,\N,\N,\N,\N
4,3742542,glurp,\N,2008-10-05 10:24:08,USR,fr,IdF,Paris,"paris, france"


In [13]:
users.location.fillna('\\N',inplace = True)
users.company.fillna('\\N',inplace = True)

In [14]:
users.created_at = pd.to_datetime(users.created_at)

In [15]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50008 entries, 0 to 50007
Data columns (total 9 columns):
id              50008 non-null int64
login           50008 non-null object
company         50008 non-null object
created_at      50008 non-null datetime64[ns]
type            50008 non-null object
country_code    50008 non-null object
state           50008 non-null object
city            50008 non-null object
location        50008 non-null object
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.4+ MB


In [17]:
users.to_pickle('data/pickled/new_subset_users.pkl')

## Create Forked repo relationships between User -> Repo

In [18]:
repos = pd.read_pickle('data/pickled/new_subset_repos.pkl')
users = pd.read_pickle('data/pickled/new_subset_users.pkl')


In [19]:
repos.info()
print "-----------------------------------"
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188865 entries, 0 to 1188864
Data columns (total 7 columns):
id             1188865 non-null int64
url            1188865 non-null object
owner_id       1188865 non-null int64
name           1188859 non-null object
language       1188865 non-null object
created_at     1188865 non-null datetime64[ns]
forked_from    1188865 non-null int64
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 63.5+ MB
-----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50008 entries, 0 to 50007
Data columns (total 9 columns):
id              50008 non-null int64
login           50008 non-null object
company         50008 non-null object
created_at      50008 non-null datetime64[ns]
type            50008 non-null object
country_code    50008 non-null object
state           50008 non-null object
city            50008 non-null object
location        50008 non-null object
dtypes: datetime64[ns](1), int64(1), object(7)


In [97]:
forked = repos[['owner_id', 'forked_from', 'created_at']]
forked = forked.rename(columns={'owner_id': 'user_id', 'forked_from': 'repo_id'})
forked['forked'] = 2
forked.head()
forked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228180 entries, 0 to 228179
Data columns (total 4 columns):
user_id       228180 non-null int64
repo_id       228180 non-null int64
created_at    228180 non-null datetime64[ns]
forked        228180 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.0 MB


In [98]:
forked = forked[forked.repo_id != 0]
forked.info()
forked.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133747 entries, 3 to 228179
Data columns (total 4 columns):
user_id       133747 non-null int64
repo_id       133747 non-null int64
created_at    133747 non-null datetime64[ns]
forked        133747 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 5.1 MB


,user_id,repo_id,created_at,forked
3,10557672,2149824,2015-11-17 16:14:36,2
8,7951407,51352312,2017-02-07 13:29:06,2
9,138931,3489262,2017-03-07 10:16:07,2
14,4611054,9805335,2017-01-23 06:15:41,2
17,2135533,51272249,2016-12-08 13:17:53,2


In [ ]:
# forked.to_pickle('data/pickled/active_forks.pkl')

## Create Owns relationship between User -> Repo
#### A user who FORKED a repo also OWNS it, but a repo that a user OWNS was not necessarily FORKED.
I don't think owning a repo versus forking a repo has any hierarchical relationship. 
So I'll use OWNS to denote a repo which was either forked or created by a user

In [3]:
repos = pd.read_pickle('data/pickled/new_subset_repos.pkl')

In [20]:
repos.head()

,id,url,owner_id,name,language,created_at,forked_from
0,2,https://api.github.com/repos/kennethkalmer/ruo...,4,ruote-kit,Ruby,2009-06-10 20:32:21,0
1,3,https://api.github.com/repos/matplotlib/basemap,23,basemap,C++,2011-02-19 02:58:42,0
2,5,https://api.github.com/repos/funkaster/cocos2d-x,28,cocos2d-x,C,2012-03-12 17:48:19,6
3,6,https://api.github.com/repos/cocos2d/cocos2d-x,31,cocos2d-x,C++,2010-11-18 23:17:00,0
4,10,https://api.github.com/repos/chapuni/llvm,67,llvm,C++,2011-02-01 09:11:48,0


In [21]:
owns = repos[['id', 'owner_id', 'forked_from', 'created_at']]
owns = owns.rename(columns={'id': 'repo_id', 'owner_id': 'user_id', 'forked_from': 'forked_from_repo_id'})
owns['owned'] = 4

In [22]:
owns.info()
owns.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188865 entries, 0 to 1188864
Data columns (total 5 columns):
repo_id                1188865 non-null int64
user_id                1188865 non-null int64
forked_from_repo_id    1188865 non-null int64
created_at             1188865 non-null datetime64[ns]
owned                  1188865 non-null int64
dtypes: datetime64[ns](1), int64(4)
memory usage: 45.4 MB


,repo_id,user_id,forked_from_repo_id,created_at,owned
0,2,4,0,2009-06-10 20:32:21,4
1,3,23,0,2011-02-19 02:58:42,4
2,5,28,6,2012-03-12 17:48:19,4
3,6,31,0,2010-11-18 23:17:00,4
4,10,67,0,2011-02-01 09:11:48,4
5,11,69,0,2011-05-16 21:41:09,4
6,19,78,0,2009-01-05 14:00:43,4
7,23,86,0,2011-08-23 02:30:04,4
8,27,89,1262879,2011-12-25 14:51:08,4
9,28,102,0,2008-11-03 11:19:43,4


In [28]:
# created_at denotes date the fork was created, not the original repo creation date
print owns.loc[owns.repo_id == 28922584]
owns.loc[owns.repo_id == 2149824]

    repo_id   user_id  forked_from_repo_id          created_at  owned
3  28922584  10557672              2149824 2015-11-17 16:14:36      4


,repo_id,user_id,forked_from_repo_id,created_at,owned
140738,2149824,149580,0,2013-01-13 18:54:59,4


In [23]:
owns.to_pickle('data/pickled/new_subset_owns.pkl')

## Create Starred relationship between User -> Repo

In [24]:
stars = pd.read_csv('data/new_subset_data/new_subset_stars.csv', sep='\t')
stars.info()
stars.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395573 entries, 0 to 2395572
Data columns (total 3 columns):
repo_id       int64
user_id       int64
created_at    object
dtypes: int64(2), object(1)
memory usage: 54.8+ MB


,repo_id,user_id,created_at
0,708,421183,2012-07-30 11:10:10
1,992,421183,2012-07-28 09:25:36
2,4077,421183,2012-08-21 08:24:15
3,4654,421183,2012-08-01 23:48:50
4,8845,421183,2012-07-30 12:57:14


In [32]:
stars.created_at = pd.to_datetime(stars.created_at)
stars['starred'] = 1

In [33]:
stars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395573 entries, 0 to 2395572
Data columns (total 4 columns):
repo_id       int64
user_id       int64
created_at    datetime64[ns]
starred       int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 73.1 MB


In [34]:
stars.to_pickle('data/pickled/new_subset_starred.pkl')

## Combine the Star & Own actions dfs to create implicit ratings
#### I'm leaving out FORKS since a fork is always owned, but and owned repo is not always a fork

In [35]:
owns = pd.read_pickle('data/pickled/new_subset_owns.pkl')
stars = pd.read_pickle('data/pickled/new_subset_starred.pkl')

In [36]:
stars.info()
owns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395573 entries, 0 to 2395572
Data columns (total 4 columns):
repo_id       int64
user_id       int64
created_at    datetime64[ns]
starred       int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 73.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188865 entries, 0 to 1188864
Data columns (total 5 columns):
repo_id                1188865 non-null int64
user_id                1188865 non-null int64
forked_from_repo_id    1188865 non-null int64
created_at             1188865 non-null datetime64[ns]
owned                  1188865 non-null int64
dtypes: datetime64[ns](1), int64(4)
memory usage: 45.4 MB


In [37]:
own_star = pd.merge(owns, stars, how='outer', on=['user_id', 'repo_id'], suffixes=('_own', '_star'))
own_star.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3540504 entries, 0 to 3540503
Data columns (total 7 columns):
repo_id                int64
user_id                int64
forked_from_repo_id    float64
created_at_own         datetime64[ns]
owned                  float64
created_at_star        datetime64[ns]
starred                float64
dtypes: datetime64[ns](2), float64(3), int64(2)
memory usage: 216.1 MB


In [38]:
own_star.head(20)

,repo_id,user_id,forked_from_repo_id,created_at_own,owned,created_at_star,starred
0,2,4,0.0,2009-06-10 20:32:21,4.0,NaT,NaN
1,3,23,0.0,2011-02-19 02:58:42,4.0,NaT,NaN
2,5,28,6.0,2012-03-12 17:48:19,4.0,NaT,NaN
3,6,31,0.0,2010-11-18 23:17:00,4.0,NaT,NaN
4,10,67,0.0,2011-02-01 09:11:48,4.0,2011-02-01 08:11:48,1.0
5,11,69,0.0,2011-05-16 21:41:09,4.0,NaT,NaN
6,19,78,0.0,2009-01-05 14:00:43,4.0,NaT,NaN
7,23,86,0.0,2011-08-23 02:30:04,4.0,NaT,NaN
8,27,89,1262879.0,2011-12-25 14:51:08,4.0,NaT,NaN
9,28,102,0.0,2008-11-03 11:19:43,4.0,NaT,NaN


In [39]:
own_star.forked_from_repo_id.fillna(0, inplace=True)
own_star.forked_from_repo_id = own_star.forked_from_repo_id.astype(np.int64)

In [40]:
own_star['interest_rank'] = own_star[['starred','owned']].sum(axis=1)
own_star.head(20)

,repo_id,user_id,forked_from_repo_id,created_at_own,owned,created_at_star,starred,interest_rank
0,2,4,0,2009-06-10 20:32:21,4.0,NaT,NaN,4.0
1,3,23,0,2011-02-19 02:58:42,4.0,NaT,NaN,4.0
2,5,28,6,2012-03-12 17:48:19,4.0,NaT,NaN,4.0
3,6,31,0,2010-11-18 23:17:00,4.0,NaT,NaN,4.0
4,10,67,0,2011-02-01 09:11:48,4.0,2011-02-01 08:11:48,1.0,5.0
5,11,69,0,2011-05-16 21:41:09,4.0,NaT,NaN,4.0
6,19,78,0,2009-01-05 14:00:43,4.0,NaT,NaN,4.0
7,23,86,0,2011-08-23 02:30:04,4.0,NaT,NaN,4.0
8,27,89,1262879,2011-12-25 14:51:08,4.0,NaT,NaN,4.0
9,28,102,0,2008-11-03 11:19:43,4.0,NaT,NaN,4.0


In [41]:
# Where repos are both owned and starred
(own_star[~own_star['starred'].isnull() & ~own_star['owned'].isnull()]).shape

(43934, 8)

In [42]:
own_star.sort_values(by=['user_id', 'repo_id']).head(10)

,repo_id,user_id,forked_from_repo_id,created_at_own,owned,created_at_star,starred,interest_rank
39758,301466,1,0,2010-02-09 11:41:12,4.0,NaT,NaN,4.0
210960,5074084,1,0,2009-12-04 09:56:46,4.0,NaT,NaN,4.0
7002,25532,2,0,2010-04-02 08:15:45,4.0,NaT,NaN,4.0
8074,28923,2,0,2008-04-10 02:07:50,4.0,NaT,NaN,4.0
13240,49907,2,0,2008-04-01 04:21:05,4.0,NaT,NaN,4.0
24994,133800,2,0,2011-02-10 08:49:30,4.0,NaT,NaN,4.0
28827,169210,2,0,2009-08-05 14:23:00,4.0,NaT,NaN,4.0
36995,258837,2,0,2008-04-21 22:57:26,4.0,NaT,NaN,4.0
50992,504851,2,0,2008-12-06 13:02:09,4.0,NaT,NaN,4.0
56529,623841,2,0,2011-11-22 11:41:06,4.0,NaT,NaN,4.0


In [43]:
own_star['created_at_star'] = own_star['created_at_star'].fillna(own_star['created_at_own'])

In [44]:
own_star.drop(['created_at_own'], axis=1, inplace=True)

In [45]:
own_star.rename(columns={'created_at_star': 'created_at'}, inplace=True)
own_star.head(10)

,repo_id,user_id,forked_from_repo_id,owned,created_at,starred,interest_rank
0,2,4,0,4.0,2009-06-10 20:32:21,NaN,4.0
1,3,23,0,4.0,2011-02-19 02:58:42,NaN,4.0
2,5,28,6,4.0,2012-03-12 17:48:19,NaN,4.0
3,6,31,0,4.0,2010-11-18 23:17:00,NaN,4.0
4,10,67,0,4.0,2011-02-01 08:11:48,1.0,5.0
5,11,69,0,4.0,2011-05-16 21:41:09,NaN,4.0
6,19,78,0,4.0,2009-01-05 14:00:43,NaN,4.0
7,23,86,0,4.0,2011-08-23 02:30:04,NaN,4.0
8,27,89,1262879,4.0,2011-12-25 14:51:08,NaN,4.0
9,28,102,0,4.0,2008-11-03 11:19:43,NaN,4.0


In [46]:
own_star.fillna(0, inplace=True)
own_star.head(20)

,repo_id,user_id,forked_from_repo_id,owned,created_at,starred,interest_rank
0,2,4,0,4.0,2009-06-10 20:32:21,0.0,4.0
1,3,23,0,4.0,2011-02-19 02:58:42,0.0,4.0
2,5,28,6,4.0,2012-03-12 17:48:19,0.0,4.0
3,6,31,0,4.0,2010-11-18 23:17:00,0.0,4.0
4,10,67,0,4.0,2011-02-01 08:11:48,1.0,5.0
5,11,69,0,4.0,2011-05-16 21:41:09,0.0,4.0
6,19,78,0,4.0,2009-01-05 14:00:43,0.0,4.0
7,23,86,0,4.0,2011-08-23 02:30:04,0.0,4.0
8,27,89,1262879,4.0,2011-12-25 14:51:08,0.0,4.0
9,28,102,0,4.0,2008-11-03 11:19:43,0.0,4.0


In [47]:
own_star.to_pickle('data/pickled/new_subset_own_and_star.pkl')

## How many repos were starred THEN forked by a user?

In [77]:
f_and_s = pd.read_pickle('data/pickled/forks_and_stars.pkl')
f_and_s.columns

Index([u'user_id', u'repo_id', u'forked', u'created_at', u'starred',
       u'interest_rank'],
      dtype='object')

In [74]:
# how many repos were starred and forked
f_and_s_eda = (f_and_s[~f_and_s['starred'].isnull() & ~f_and_s['forked'].isnull()]).copy()
f_and_s_eda.shape[0]

1016532

In [75]:
#Number of repos starred AFTER they were forked by a user
(f_and_s_eda[f_and_s_eda.created_at_star >= f_and_s_eda.created_at_fork]).shape[0]

AttributeError: 'DataFrame' object has no attribute 'created_at_star'

In [76]:
#Number of repos starred BEFORE they were forked by a user
(f_and_s_eda[f_and_s_eda.created_at_star < f_and_s_eda.created_at_fork]).shape[0]

AttributeError: 'DataFrame' object has no attribute 'created_at_star'

In [167]:
f_and_s_eda.sort_values(by=['user_id', 'repo_id'])

,user_id,repo_id,created_at_fork,forked,created_at_star,starred,interest_rank
28052,10,20078281,2015-05-25 14:06:50,2.0,2014-11-26 19:57:11,1.0,3.0
78424,10,43644249,2017-05-18 16:07:18,2.0,2017-01-16 09:21:27,1.0,3.0
855,30,6,2011-03-16 02:46:22,2.0,2010-11-19 08:06:45,1.0,3.0
361,32,6,2011-09-19 03:20:38,2.0,2010-11-23 12:02:36,1.0,3.0
39802,107,13860,2015-12-04 17:57:32,2.0,2014-10-09 06:47:45,1.0,3.0
22713,203,10313229,2014-01-22 05:29:04,2.0,2010-06-16 18:39:03,1.0,3.0
24038,245,10887897,2017-04-11 05:30:13,2.0,2017-04-11 05:26:04,1.0,3.0
90113,283,6830,2017-05-25 06:38:36,2.0,2015-10-10 08:52:43,1.0,3.0
44249,300,205522,2017-03-14 10:43:09,2.0,2011-07-02 09:07:47,1.0,3.0
133745,418,13758182,2017-03-02 23:13:52,2.0,2016-07-03 22:25:22,1.0,3.0


In [185]:
f_and_s_eda[f_and_s_eda.created_at_star >= f_and_s_eda.created_at_fork]

,user_id,repo_id,created_at_fork,forked,created_at_star,starred,interest_rank
14,809492,19091882,2015-06-05 22:12:32,2.0,2016-06-17 01:19:49,1.0,3.0
26,3351,7704341,2014-02-28 23:17:54,2.0,2016-11-16 07:11:46,1.0,3.0
35,2381056,21382248,2015-10-06 16:36:40,2.0,2015-10-17 03:30:19,1.0,3.0
44,730680,8206594,2017-03-18 13:46:54,2.0,2017-03-24 13:21:32,1.0,3.0
62,3925561,10739107,2014-07-14 22:08:29,2.0,2015-10-12 17:50:58,1.0,3.0
63,1961499,898600,2013-07-24 08:40:55,2.0,2014-05-31 17:03:26,1.0,3.0
77,7121874,27842288,2015-10-27 13:25:02,2.0,2017-01-19 11:06:37,1.0,3.0
90,626216,13155973,2014-11-05 15:24:41,2.0,2015-06-15 18:46:17,1.0,3.0
91,9525404,56189053,2017-02-02 14:39:33,2.0,2017-04-21 03:27:01,1.0,3.0
136,6509770,24311289,2015-12-29 02:13:54,2.0,2016-01-31 18:05:51,1.0,3.0


In [225]:
f_and_s_eda['time_star2fork'] = f_and_s_eda['created_at_fork'].subtract(
    f_and_s_eda['created_at_star'])
f_and_s_eda['time_star2fork_int'] = pd.to_numeric(f_and_s_eda['time_star2fork'])
f_and_s_eda.sort_values(by=['user_id', 'repo_id']).head(10)

,user_id,repo_id,created_at_fork,forked,created_at_star,starred,interest_rank,time_star2fork,time_star2fork_int
28052,10,20078281,2015-05-25 14:06:50,2.0,2014-11-26 19:57:11,1.0,3.0,179 days 18:09:39,15530979000000000
78424,10,43644249,2017-05-18 16:07:18,2.0,2017-01-16 09:21:27,1.0,3.0,122 days 06:45:51,10565151000000000
855,30,6,2011-03-16 02:46:22,2.0,2010-11-19 08:06:45,1.0,3.0,116 days 18:39:37,10089577000000000
361,32,6,2011-09-19 03:20:38,2.0,2010-11-23 12:02:36,1.0,3.0,299 days 15:18:02,25888682000000000
39802,107,13860,2015-12-04 17:57:32,2.0,2014-10-09 06:47:45,1.0,3.0,421 days 11:09:47,36414587000000000
22713,203,10313229,2014-01-22 05:29:04,2.0,2010-06-16 18:39:03,1.0,3.0,1315 days 10:50:01,113655001000000000
24038,245,10887897,2017-04-11 05:30:13,2.0,2017-04-11 05:26:04,1.0,3.0,0 days 00:04:09,249000000000
90113,283,6830,2017-05-25 06:38:36,2.0,2015-10-10 08:52:43,1.0,3.0,592 days 21:45:53,51227153000000000
44249,300,205522,2017-03-14 10:43:09,2.0,2011-07-02 09:07:47,1.0,3.0,2082 days 01:35:22,179890522000000000
133745,418,13758182,2017-03-02 23:13:52,2.0,2016-07-03 22:25:22,1.0,3.0,242 days 00:48:30,20911710000000000


In [72]:
plt.figure(figsize=(10,5))
plt.hist(f_and_s_eda['time_star2fork_int'], bins=100)
plt.title("Repos Both Starred and Forked")
plt.xlabel("Time From Star and Fork (sec)")
plt.ylabel("User-to-Repo Interactions");
plt.xlim((-1e17,1e17));

NameError: name 'f_and_s_eda' is not defined

## Create a user-to-repo lookup dataframe

In [102]:
repos = pd.read_pickle('data/pickled/active_repos.pkl')
users = pd.read_pickle('data/pickled/active_users.pkl')

In [103]:
from itertools import product


In [105]:
user_repo_lookup = pd.DataFrame(list(product(users['id'], repos['id'])), columns=['user_id', 'repo_id'])
user_repo_lookup.head(50)

MemoryError: 

In [54]:
# Nevermind. Too big of a dataframe :/